# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Setup

### Imports

In [1]:
import os
import json
import warnings
from os.path import join
from tqdm.notebook import tqdm
from itertools import repeat, starmap, product

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from numpy.linalg import norm
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from sklearn.model_selection import StratifiedGroupKFold
from kagglehub import whoami, competition_download, dataset_upload

### Supress performance warngings

In [2]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### Config

In [3]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
GRAVITY_WORLD = np.array([0, 0, 9.81], "float32")
RAW_ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
LINEAR_ACC_COLS = ["gravity_free_" + col for col in RAW_ACCELRATION_COLS]
COMPETITION_HANDLE = "cmi-detect-behavior-with-sensor-data"
CATEGORY_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
DATASET_DF_DTYPES = {
    "acc_x": "float32", "acc_y": "float32", "acc_z": "float32",
    "thm_1":"float32", "thm_2":"float32", "thm_3":"float32", "thm_4":"float32", "thm_5":"float32",
    "sequence_counter": "int32",
    **{col: "category" for col in CATEGORY_COLUMNS},
    **{f"tof_{i_1}_v{i_2}": "float32" for i_1, i_2 in product(range(1, 5), range(64))},
}
PREPROCESSED_DATASET_HANDLE = "mauroabidalcarrer/prepocessed-cmi-2025"
# The quantile of the sequences len used to pad/truncate during preprocessing
SEQUENCE_NORMED_LEN_QUANTILE = 0.95
# SAMPLING_FREQUENCY = 10 #Hz
N_FOLDS = 5
VALIDATION_FRACTION = 0.2
TARGET_NAMES = [
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
]
EPSILON=1e-8
DELTA_ROTATION_ANGULAR_VELOCITY_COLS = ["angular_vel_x", "angular_vel_y", "angular_vel_z"]
DELTA_ROTATION_AXES_COLS = ["rotation_axis_x", "rotation_axis_y", "rotation_axis_z"]
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [4]:
def get_feature_cols(df:DF) -> list[str]:
    return sorted(list(set(df.columns) - set(META_DATA_COLUMNS) - set(TARGET_NAMES)))

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [5]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

## Data preprocessing

### Impute missing data
Perform forward, backward and then filling of all NaN sequences.

In [6]:
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.
def get_fillna_val_per_feature_col(df:DF) -> dict:
    return {col: 1.0 if col == 'rot_w' else 0 for col in get_feature_cols(df)}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    # fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(get_fillna_val_per_feature_col(df))
    )
    return df

df = imputed_features(df)

### Norm quaternions
This allows us to parse quaternions with `scipy.spatial.transform.Rotation`.

In [7]:
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Linear acceleration
Remove gravity from the acceleration features.

In [8]:
def add_linear_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations:Rotation = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True).astype("float32")
    df[LINEAR_ACC_COLS] = df[RAW_ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_linear_acc_cols(df)

### Accelerations magnitudes
Add magnitue (norm) of both  raw acceleration and linear accelration.

In [9]:
def add_acc_magnitude(df:DF, acc_cols:list[str], acc_mag_col_name:str) -> DF:
    return df.assign(**{acc_mag_col_name: np.linalg.norm(df.loc[:, acc_cols], axis=1)})

df = add_acc_magnitude(df, RAW_ACCELRATION_COLS, "raw_acc_mag")
df = add_acc_magnitude(df, LINEAR_ACC_COLS, "linear_acc_mag")

### Quaternion angle magnitude
Refered to as "rotation angle" in top notebooks.

In [10]:
def add_quat_angle_mag(df:DF) -> DF:
    return df.assign(quat_rot_mag=np.arccos(df["rot_w"]) * 2)

df = add_quat_angle_mag(df)

### Angular velocity
Compute the axis of the rotation difference and its angular magnutide.  
Top notebooks scale the vector by the angular magnitude.  
I will also include the magnitude and unit axis vector in case they turn out to be more informative.   

In [11]:
def add_angular_velocity_features(df:DF) -> DF:
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    delta_rotations = rotations[1:] * rotations[:-1].inv()
    delta_rot_velocity = delta_rotations.as_rotvec()
    # Add extra line to avoid shape mismatch
    delta_rot_velocity = np.vstack((np.zeros((1, 3)), delta_rot_velocity))
    delta_rot_magnitude = norm(delta_rot_velocity, axis=1, keepdims=True)
    delta_rot_axes = delta_rot_velocity / (delta_rot_magnitude + EPSILON)
    df[DELTA_ROTATION_ANGULAR_VELOCITY_COLS] = delta_rot_velocity
    df[DELTA_ROTATION_AXES_COLS] = delta_rot_axes
    df["delta_rot_mag"] = delta_rot_magnitude.squeeze()

    return df

df = add_angular_velocity_features(df)

### Euler angles from quaternions

In [12]:
def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df

df = rot_euler_angles(df)

### One hot encode target values.

In [13]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [14]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Add derivatives w.r.t time features

In [15]:
def add_diff_features(df:DF) -> DF:
    df[[col + "_diff" for col in get_feature_cols(df)]] = (
        df
        .groupby("sequence_id", observed=True)
        [get_feature_cols(df)]
        .diff()
        .fillna(get_fillna_val_per_feature_col(df))
        .values
    )
    return df

df = add_diff_features(df)

In [16]:
len(get_feature_cols(df))

66

### Split into folds

In [17]:
df["gesture"].unique().tolist()

['Cheek - pinch skin',
 'Forehead - pull hairline',
 'Write name on leg',
 'Feel around in tray and pull out an object',
 'Neck - scratch',
 'Neck - pinch skin',
 'Eyelash - pull hair',
 'Eyebrow - pull hair',
 'Forehead - scratch',
 'Above ear - pull hair',
 'Wave hello',
 'Write name in air',
 'Text on phone',
 'Pull air toward your face',
 'Pinch knee/leg skin',
 'Scratch knee/leg skin',
 'Drink from bottle/cup',
 'Glasses on/off']

In [18]:
def split_dataset_stratified_groupkfold(df: pd.DataFrame, target_col: str, group_col: str) -> list[tuple[pd.DataFrame, pd.DataFrame]]:
    sgkf = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True)

    folds = []
    X = df.drop(columns=[target_col])
    y = df[target_col]
    groups = df[group_col]

    for train_idx, valid_idx in sgkf.split(X, y, groups):
        train_df = df.iloc[train_idx].copy()
        valid_df = df.iloc[valid_idx].copy()
        folds.append((train_df, valid_df))

    return folds

# Example usage:
folds = split_dataset_stratified_groupkfold(df, target_col="gesture", group_col="subject")

### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [19]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[get_feature_cols(df)].mean().astype("float32")
    stds = train[get_feature_cols(df)].std().astype("float32")
    train.loc[:, get_feature_cols(df)] = (train[get_feature_cols(df)] - means) / stds
    val.loc[:, get_feature_cols(df)] = (val[get_feature_cols(df)] - means) / stds
    return train, val

folds = list(starmap(std_norm_dataset, folds))

Normalize full dataset.

In [20]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[get_feature_cols(df)].mean().astype("float32").to_dict(),
    "std": df[get_feature_cols(df)].std().astype("float32").to_dict(),
}
df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

/tmp/ipykernel_7687/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.87240755 0.91835342 0.70619159 ... 0.24943555 0.40213801 0.45484063]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_7687/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.8841304   0.8841304   0.72331979 ... -0.31414283 -0.42967666
 -0.54442986]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_7687/629001182.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated an

Verify the mean and std of the full dataset.

In [21]:
df[get_feature_cols(df)].agg(["mean", "std"])

,acc_x,acc_x_diff,acc_y,acc_y_diff,acc_z,acc_z_diff,angular_vel_x,angular_vel_x_diff,angular_vel_y,angular_vel_y_diff,...,tof_1,tof_1_diff,tof_2,tof_2_diff,tof_3,tof_3_diff,tof_4,tof_4_diff,tof_5,tof_5_diff
mean,-1.749467e-08,-3.192562e-10,1.186949e-08,-1.666452e-10,-2.971537e-09,-3.622585e-10,8.305834e-10,4.009440e-10,-5.170893e-10,-1.785537e-10,...,1.908273e-07,-2.311552e-10,1.687596e-08,8.222476e-10,2.663846e-09,-6.040385e-10,1.404808e-08,6.444108e-11,-5.993022e-08,-8.028372e-11
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [22]:
pd.concat({
    "train": folds[0][0][get_feature_cols(df)].agg(["mean", "std"]),
    "validation": folds[0][1][get_feature_cols(df)].agg(["mean", "std"]),
})

acc_x    acc_x_diff         acc_y    acc_y_diff  \
train      mean -3.960707e-09 -7.119836e-10 -2.587662e-08 -1.499695e-10   
           std   9.999999e-01  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -1.151082e-02 -1.362219e-03  7.247336e-02  1.321800e-03   
           std   1.006927e+00  9.908973e-01  9.593747e-01  9.422414e-01   

                        acc_z    acc_z_diff  angular_vel_x  \
train      mean -1.901139e-08  1.550206e-09   1.647441e-10   
           std   1.000000e+00  1.000000e+00   1.000000e+00   
validation mean -2.688467e-02  6.080548e-03   3.185224e-03   
           std   9.903418e-01  9.999893e-01   1.005739e+00   

                 angular_vel_x_diff  angular_vel_y  angular_vel_y_diff  ...  \
train      mean       -2.924493e-10   2.390111e-10        3.607899e-10  ...   
           std         1.000000e+00   1.000000e+00        1.000000e+00  ...   
validation mean       -1.696636e-03   1.361521e-02       -6.836507e-03  ...   
           std         1.026171e+00   1.036483e+00        1.039976e+00  ...   

                        tof_1    tof_1_diff         tof_2    tof_2_diff  \
train      mean -7.261296e-08  3.671856e-10 -1.241022e-08  1.935463e-10   
           std   1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -1.762115e-01  1.460750e-03 -1.797978e-01  2.075302e-03   
           std   1.134178e+00  1.010892e+00  1.102369e+00  9.921692e-01   

                        tof_3    tof_3_diff         tof_4    tof_4_diff  \
train      mean -5.544990e-08  2.354477e-10 -1.953949e-08  4.316228e-10   
           std   1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -1.596343e-01  3.929143e-03 -1.796857e-01  1.246480e-03   
           std   1.112410e+00  1.016164e+00  1.152484e+00  9.828607e-01   

                        tof_5    tof_5_diff  
train      mean -6.020274e-08 -7.631861e-12  
           std   9.999999e-01  1.000000e+00  
validation mean -4.049730e-02  3.301809e-03  
           std   9.982337e-01  9.963560e-01  

[4 rows x 66 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [23]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [24]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [25]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 130
validation normed sequence len: 116

train normed sequence len: 129
validation normed sequence len: 118

train normed sequence len: 127
validation normed sequence len: 119

train normed sequence len: 122
validation normed sequence len: 146

train normed sequence len: 125
validation normed sequence len: 133



#### Sequence length norm implementation

In [26]:
gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, get_feature_cols(df)]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(get_feature_cols(df))),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarrays(train, normed_sequence_len),
        *df_to_ndarrays(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6520 [00:00<?, ?it/s]

  0%|          | 0/1631 [00:00<?, ?it/s]

  0%|          | 0/6417 [00:00<?, ?it/s]

  0%|          | 0/1734 [00:00<?, ?it/s]

  0%|          | 0/6581 [00:00<?, ?it/s]

  0%|          | 0/1570 [00:00<?, ?it/s]

  0%|          | 0/6567 [00:00<?, ?it/s]

  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

In [27]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [ ]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
full_dataset_meta_data["target_names"] = one_hot_target.columns.to_list()
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["feature_cols"] = get_feature_cols(df)

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [30]:
df[get_feature_cols(df)]

,acc_x,acc_x_diff,acc_y,acc_y_diff,acc_z,acc_z_diff,angular_vel_x,angular_vel_x_diff,angular_vel_y,angular_vel_y_diff,...,tof_1,tof_1_diff,tof_2,tof_2_diff,tof_3,tof_3_diff,tof_4,tof_4_diff,tof_5,tof_5_diff
0,0.872408,0.015724,0.884130,-0.027387,0.625816,0.036749,-0.021278,-0.011243,-0.010010,-0.007949,...,0.339571,0.022522,0.148727,0.026558,-0.454938,0.034665,-0.016778,0.018485,0.404678,0.004162
1,0.918353,0.139234,0.884130,-0.027387,0.588012,-0.075917,-0.075474,-0.067267,-0.238310,-0.213256,...,0.349948,0.089104,0.199531,0.363426,-0.337148,0.824043,0.002596,0.140912,0.433771,0.218816
2,0.706192,-0.554598,0.723320,-0.549341,0.964766,1.159588,-0.554449,-0.506371,-1.075163,-0.760519,...,0.398864,0.336407,0.371769,1.168622,0.022423,2.444347,0.026308,0.168322,0.490583,0.423344
3,0.858218,0.424395,0.347835,-1.246124,1.134561,0.542790,-0.961076,-0.431583,-0.665305,0.360630,...,0.621508,1.451174,0.644067,1.832088,0.543179,3.524549,0.459197,2.753924,0.532300,0.311967
4,0.679165,-0.465599,-0.302433,-2.138006,1.655481,1.589246,-0.618189,0.343207,-1.052999,-0.356597,...,1.073910,2.925483,0.789974,0.994026,0.870276,2.226724,0.731018,1.736129,0.684347,1.126032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,0.322408,-0.020602,-0.444509,0.172780,-1.309212,0.409119,-0.231978,-0.291376,0.171270,0.428464,...,-1.001627,-0.004111,-0.829872,0.186776,0.541703,0.155347,-0.929112,0.404041,-1.044439,0.334243
574941,0.369030,0.141050,-0.490566,-0.176879,-1.434796,-0.337531,-0.119604,0.104920,-0.136954,-0.285131,...,-0.969906,0.226072,-0.851557,-0.117227,0.533437,-0.020730,-0.996778,-0.409099,-1.014798,0.222866
574942,0.249436,-0.305763,-0.314143,0.545242,-1.138775,0.918980,-0.010458,0.101584,-0.038358,0.080717,...,-1.037499,-0.411210,-0.925905,-0.466419,0.399410,-0.863526,-1.002272,-0.016234,-0.963200,0.384869
574943,0.402138,0.426211,-0.429677,-0.402383,-1.414933,-0.786284,-0.118555,-0.122986,-0.138399,-0.097915,...,-1.027123,0.089104,-0.894307,0.236073,0.467900,0.493652,-0.900484,0.661688,-1.007662,-0.323895


## Dataset upload
Optionally upload the dataset to kaggle.

In [ ]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
